# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])
nltk.download('averaged_perceptron_tagger')
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
# Global varibales
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [5]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('select * from message_processed',engine)
X = df['message']
Y = df.iloc[:,4:]

In [6]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
id                        26216 non-null int64
message                   26216 non-null object
original                  10170 non-null object
genre                     26216 non-null object
related                   26216 non-null int64
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null i

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
# Split train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y,random_state=42)

In [11]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
# predict on test data
y_pred = pipeline.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print(classification_report(y_test[column],y_pred_df[column]))

             precision    recall  f1-score   support

          0       0.59      0.34      0.43      1563
          1       0.81      0.92      0.86      4944
          2       0.83      0.11      0.19        47

avg / total       0.76      0.78      0.76      6554

             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5443
          1       0.85      0.35      0.49      1111

avg / total       0.88      0.88      0.86      6554

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

             precision    recall  f1-score   support

          0       0.73      0.88      0.80      3884
          1       0.75      0.51      0.61      2670

avg / total       0.74      0.73      0.72      6554

             precision    recall  f1-score   support

          0       0.92      1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
# Average overall accuracy
overall_accuracy = (y_pred == y_test).mean().mean()
print("Average overall accuracy is {0:.2f}%".format(overall_accuracy * 100))

Average overall accuracy is 94.17%


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
parameters = {
    'vect__ngram_range': [(1, 1)],
    'clf__estimator__n_estimators': [3, 5],
    #'clf__estimator__min_samples_split': [2, 3, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters,cv=5)
cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': [(1, 1)], 'clf__estimator__n_estimators': [3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
# predict on test data
y_pred_improve = cv.predict(X_test)
y_pred_improve_df = pd.DataFrame(y_pred_improve, columns = y_test.columns)
for column in y_test.columns:
    print(classification_report(y_test[column],y_pred_improve_df[column]))

             precision    recall  f1-score   support

          0       0.56      0.31      0.40      1563
          1       0.80      0.92      0.86      4944
          2       0.67      0.13      0.21        47

avg / total       0.74      0.77      0.75      6554

             precision    recall  f1-score   support

          0       0.90      0.96      0.93      5443
          1       0.71      0.45      0.55      1111

avg / total       0.86      0.88      0.86      6554

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6521
          1       1.00      0.03      0.06        33

avg / total       1.00      1.00      0.99      6554

             precision    recall  f1-score   support

          0       0.74      0.80      0.77      3884
          1       0.67      0.58      0.62      2670

avg / total       0.71      0.71      0.71      6554

             precision    recall  f1-score   support

          0       0.93      0.99 

In [22]:
# Average overall accuracy
overall_accuracy_improve = (y_pred_improve == y_test).mean().mean()
print("Average overall accuracy is {0:.2f}%".format(overall_accuracy_improve * 100))

Average overall accuracy is 93.96%


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    parameters = {
        'features__text_pipeline__vect__ngram_range': [(1, 1)],
        'clf__estimator__n_estimators': [3, 5],
        #'clf__estimator__min_samples_split': [2, 3, 4]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

#Fit model
model = build_model()
model.fit(X_train, y_train)
print("\nBest Parameters:", model.best_params_)


Best Parameters: {'clf__estimator__n_estimators': 5, 'features__text_pipeline__vect__ngram_range': (1, 1)}


In [25]:
# predict on test data
y_pred_improve_final = model.predict(X_test)
y_pred_improve_final_df = pd.DataFrame(y_pred_improve_final, columns = y_test.columns)
for column in y_test.columns:
    print(classification_report(y_test[column],y_pred_improve_final_df[column]))

             precision    recall  f1-score   support

          0       0.57      0.31      0.40      1563
          1       0.80      0.93      0.86      4944
          2       0.86      0.13      0.22        47

avg / total       0.75      0.77      0.75      6554

             precision    recall  f1-score   support

          0       0.89      0.97      0.93      5443
          1       0.72      0.42      0.53      1111

avg / total       0.86      0.87      0.86      6554

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

             precision    recall  f1-score   support

          0       0.74      0.80      0.77      3884
          1       0.67      0.58      0.62      2670

avg / total       0.71      0.71      0.71      6554

             precision    recall  f1-score   support

          0       0.93      0.99 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
# Average overall accuracy
overall_accuracy_improve = (y_pred_improve_final == y_test).mean().mean()
print("Average overall accuracy is {0:.2f}%".format(overall_accuracy_improve * 100))

Average overall accuracy is 93.99%


### 9. Export your model as a pickle file

In [27]:
with open('ml_pipeline_model.pkl','wb') as f:
    pickle.dump(pipeline,f)
print('Save ml_pipeline_model successfully!!!')

Save ml_pipeline_model successfully!!!


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.